<a href="https://colab.research.google.com/github/Sohaib9920/Transformer_Multilingual_NER/blob/main/Transformer_Multilingual_Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task:** Perform NER for a customer based in Switzerland, where there are four national languages: German (62.9%), French (22.9%), Italian (8.4%), and English (5.9%)

In [1]:
!pip install -U datasets
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.0 MB/s eta 0:00:00


In [265]:
import datasets
import transformers
from collections import defaultdict
import pandas as pd
from collections import Counter

# Dataset

We will be using a subset of the Cross-lingual TRansfer Evaluation of
Multilingual Encoders (XTREME) benchmark called WikiANN or PAN-X

In [3]:
xtreme_subsets = datasets.get_dataset_config_names("xtreme")
PAN_subsets = [s for s in xtreme_subsets if s.startswith("P")]
print(PAN_subsets)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for xtreme contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xtreme
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de', 'PAN-X.el', 'PAN-X.en', 'PAN-X.es', 'PAN-X.et', 'PAN-X.eu', 'PAN-X.fa', 'PAN-X.fi', 'PAN-X.fr', 'PAN-X.he', 'PAN-X.hi', 'PAN-X.hu', 'PAN-X.id', 'PAN-X.it', 'PAN-X.ja', 'PAN-X.jv', 'PAN-X.ka', 'PAN-X.kk', 'PAN-X.ko', 'PAN-X.ml', 'PAN-X.mr', 'PAN-X.ms', 'PAN-X.my', 'PAN-X.nl', 'PAN-X.pt', 'PAN-X.ru', 'PAN-X.sw', 'PAN-X.ta', 'PAN-X.te', 'PAN-X.th', 'PAN-X.tl', 'PAN-X.tr', 'PAN-X.ur', 'PAN-X.vi', 'PAN-X.yo', 'PAN-X.zh', 'PAWS-X.de', 'PAWS-X.en', 'PAWS-X.es', 'PAWS-X.fr', 'PAWS-X.ja', 'PAWS-X.ko', 'PAWS-X.zh']


In [123]:
langs = ["de", "fr", "it", "en"]
props = [0.629, 0.229, 0.084, 0.058]
panx = defaultdict(dict)
for lang, prop in zip(langs, props):
    data_dict = datasets.load_dataset("xtreme", name=f"PAN-X.{lang}")
    for split in data_dict:
        selected_size = int(prop * data_dict[split].num_rows)
        panx[lang][split] = data_dict[split].shuffle(seed=42).select(range(selected_size))
        panx[lang] = datasets.DatasetDict(panx[lang])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for xtreme contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xtreme
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [131]:
df = pd.DataFrame({lang: panx[lang].num_rows for lang in langs})
df

,de,fr,it,en
train,12580,4580,1680,1160
validation,6290,2290,840,580
test,6290,2290,840,580


In [135]:
df.sum(axis=1)

train         20000
validation    10000
test          10000
dtype: int64

In [210]:
tags = panx["de"]["train"].features["ner_tags"].feature
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

Each article is annotated with LOC (location), PER
(person), and ORG (organization) tags in the “inside-outside-beginning” (IOB2) format. In this format, a B-prefix indicates the beginning of an entity, and consecutive
tokens belonging to the same entity are given an I-prefix. An O tag indicates that the
token does not belong to any entity

In [215]:
panx_de = panx["de"].map(lambda batch: {"ner_tags_str":tags.int2str(batch["ner_tags"])})

In [228]:
pd.set_option("display.max_colwidth", None)
pd.DataFrame(panx_de["train"]).head(5)

,tokens,ner_tags,langs,ner_tags_str
0,"[Olympique, Nîmes, ,, Auxerres, seinerzeitiger, drittklassiger, Endspielgegner, ,, hatte, sich, erst, gar, nicht, für, die, Hauptrunde, qualifizieren, können, .]","[3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[de, de, de, de, de, de, de, de, de, de, de, de, de, de, de, de, de, de, de]","[B-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,"[Alexander, Jakowlew, wies, deshalb, am, 18, .]","[1, 2, 0, 0, 0, 0, 0]","[de, de, de, de, de, de, de]","[B-PER, I-PER, O, O, O, O, O]"
2,"[Aktuell, werden, die, beiden, am, besten, erhaltenen, Tafeln, im, Alten, Museum, ausgestellt, .]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0]","[de, de, de, de, de, de, de, de, de, de, de, de, de]","[O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O]"
3,"[Sie, gehören, damit, mit, der, Dickschnabellumme, zu, den, schwersten, ,, derzeit, noch, lebenden, Alkenarten, .]","[0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[de, de, de, de, de, de, de, de, de, de, de, de, de, de, de]","[O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O]"
4,"[Auch, in, Garmisch-Partenkirchen, landete, er, auf, diesem, Platz, .]","[0, 0, 5, 0, 0, 0, 0, 0, 0]","[de, de, de, de, de, de, de, de, de]","[O, O, B-LOC, O, O, O, O, O, O]"


In [257]:
df = pd.DataFrame()
for split in panx_de:
    count = Counter()
    for row in panx_de[split]["ner_tags_str"]:
        count.update(row)
    df[split] = dict(count)

In [259]:
df.T

,B-ORG,I-ORG,O,B-PER,I-PER,B-LOC,I-LOC
train,5397,7647,86156,5881,8423,6169,2898
validation,2639,3774,43122,2870,4140,3172,1447
test,2657,3825,43170,2971,4260,3100,1404


This looks good—the distributions of the PER, LOC, and ORG frequencies are roughly
the same for each split, so the validation and test sets should provide a good measure
of our NER tagger’s ability to generalize.

# Tokenization

In [266]:
bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = transformers.AutoTokenizer.from_pretrained(xlmr_model_name)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [291]:
text = "Transformer models are cool n awsome!"
print(bert_tokenizer(text).tokens())
print(xlmr_tokenizer(text).tokens())

['[CLS]', 'Trans', '##former', 'models', 'are', 'cool', 'n', 'a', '##ws', '##ome', '!', '[SEP]']
['<s>', '▁Trans', 'former', '▁models', '▁are', '▁cool', '▁n', '▁a', 'w', 'some', '!', '</s>']


XML-R uses SentencePiece instead of WordPiece used by BERT. It is suitable for multilingual corpora.